In [1]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scienceplots
import shap

import events_package.utils as utils
from events_package.Experiment import Experiment
from events_package.config import FIVE_LAYERS
from events_package.input_getters import get_Y_1, get_X_5

In [2]:
Experiment.__version__

'5.0'

# 1. Importing Single Particle Data

## 1.1 Electrons

In [3]:
# importing data
dataset_df = pd.read_parquet(
    r"C:\Users\User1\Desktop\MSci_Project\Data\6_data\Electron\Parquet\1m_electron_pq_3"
)

electrons = Experiment(dataset_df, config=FIVE_LAYERS)
del dataset_df
electrons.length

444840

In [4]:
electrons.standard_procedure()

INFO: Removed duplicates
INFO: Denoisified the dataset
INFO: Shuffled dataset
INFO: Number of events after removing duplicates: 434998
INFO: Removed events with 0 energy in layers after denoisifying
INFO: Number of events after removing 0 energy (in calorimeters) events: 434998


## 1.2 Photons

In [5]:
dataset_df = pd.read_parquet(
    r"C:\Users\User1\Desktop\MSci_Project\Data\6_data\Photon\Parquet\1m_photon_pq"
)

photons = Experiment(dataset_df, config=FIVE_LAYERS)
del dataset_df
photons.length

444142

In [6]:
photons.standard_procedure()

INFO: Removed duplicates
INFO: Denoisified the dataset
INFO: Shuffled dataset
INFO: Number of events after removing duplicates: 434870
INFO: Removed events with 0 energy in layers after denoisifying
INFO: Number of events after removing 0 energy (in calorimeters) events: 434870


## 1.3 Neutral Pions

In [7]:
dataset_df = pd.read_parquet(
    r"C:\Users\User1\Desktop\MSci_Project\Data\6_data\PiZero\Parquet\pq_pi0_2"
)

pi0 = Experiment(dataset_df, config=FIVE_LAYERS)
del dataset_df
pi0.length

412856

In [8]:
pi0.standard_procedure()

INFO: Removed duplicates
INFO: Denoisified the dataset
INFO: Shuffled dataset
INFO: Number of events after removing duplicates: 391483
INFO: Removed events with 0 energy in layers after denoisifying
INFO: Number of events after removing 0 energy (in calorimeters) events: 391483


## 1.4 Charged Pions

In [9]:
dataset_df = pd.read_parquet(
    r"C:\Users\User1\Desktop\MSci_Project\Data\6_data\PiPlusMinus\Parquet\pq_piplusminus_2"
)

pi_char = Experiment(dataset_df, config=FIVE_LAYERS)
del dataset_df
pi_char.length

357554

In [10]:
pi_char.standard_procedure()

INFO: Removed duplicates
INFO: Denoisified the dataset
INFO: Shuffled dataset
INFO: Number of events after removing duplicates: 330809
INFO: Removed events with 0 energy in layers after denoisifying
INFO: Number of events after removing 0 energy (in calorimeters) events: 330803


## 1.5 Combining Different Particle Data

In [11]:
# add types to allow for identification later
electrons.add_physics_object_type(typ="electron")
photons.add_physics_object_type(typ="photon")
pi0.add_physics_object_type(typ="pi0")
pi_char.add_physics_object_type(typ="pi_char")

In [12]:
experiment = electrons + photons + pi0 + pi_char

In [13]:
# all previous datasets have already been denoisified, duplicates were removed, no need to do it now
# in fact, doing it would delete some good events
experiment.shuffle_dataset(repeats=11)
print(experiment.length)

1592154


In [14]:
# split data into training and testing, next train XGBoost model
experiment.train_test_split(get_X=get_X_5, get_Y=get_Y_1, test_size=0.2)
print(experiment.X_test.shape)

params = {
    "objective": "reg:squarederror",
    "max_depth": 6,
    "learning_rate": 0.18,
    # "subsample": 0.8,
    "colsample_bytree": 0.8,
    "eval_metric": "rmse",
    "n_estimators": 600,
}


experiment.train_xgboost_model(params)

(318431, 22)


KeyboardInterrupt: 

In [ ]:
utils.count_nodes(experiment.model)

# Hyperparameter Scan in Search of Smaller Model

In cells below, tables represent models trained with different hyperparameters: number of rounds (trees), max depth, colsumple by tree and learnign rate.

In the end, best model with < 2000 nodes was the one with 130 trees, max depth of 4, colsample of 1.0 and a surprisingly high learning rate of 0.74 (although there was no strong dependency on this parameter, compared to for example no trees or depth). this miniutarised model has 1950 nodes in total.

In [ ]:
max_depth_range = [6]
learning_rate_range = [0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2]
colsample_bytree_range = [0.6, 0.7, 0.8, 0.9, 1.0]
num_rounds_grid = [600]

# Generate all combinations of hyperparameters
param_combinations = list(
    itertools.product(
        max_depth_range, learning_rate_range, colsample_bytree_range, num_rounds_grid
    )
)

# Construct the param_grid
param_grid = [
    {
        "objective": "reg:squarederror",
        "max_depth": max_depth,
        "learning_rate": learning_rate,
        "colsample_bytree": colsample_bytree,
        "n_estimators": n_estimators,
    }
    for max_depth, learning_rate, colsample_bytree, n_estimators in param_combinations
]


# Perform hyperparameter scan
experiment_hyperparams1 = experiment.xgboost_hyperparameter_scan(
    param_grid, nodes_info=True
)

In [ ]:
experiment_hyperparams1.sort_values(by="MSE")

In [ ]:
max_depth_range = [3, 4, 5, 6]
learning_rate_range = [0.18]
colsample_bytree_range = [0.8]
num_rounds_grid = [130, 150, 200, 300, 400, 500, 600]

# Generate all combinations of hyperparameters
param_combinations = list(
    itertools.product(
        max_depth_range, learning_rate_range, colsample_bytree_range, num_rounds_grid
    )
)

# Construct the param_grid
param_grid = [
    {
        "objective": "reg:squarederror",
        "max_depth": max_depth,
        "learning_rate": learning_rate,
        "colsample_bytree": colsample_bytree,
        "n_estimators": n_estimators,
    }
    for max_depth, learning_rate, colsample_bytree, n_estimators in param_combinations
]


# Perform hyperparameter scan
experiment_hyperparams2 = experiment.xgboost_hyperparameter_scan(
    param_grid, nodes_info=True
)

In [ ]:
max_depth_range = [4]
learning_rate_range = [
    0.1,
    0.11,
    0.12,
    0.13,
    0.14,
    0.15,
    0.16,
    0.17,
    0.18,
    0.19,
    0.2,
    0.21,
    0.22,
    0.23,
    0.24,
    0.25,
    0.26,
    0.27,
    0.28,
    0.29,
    0.30,
    0.31,
    0.32,
    0.33,
    0.34,
    0.35,
    0.36,
    0.37,
    0.38,
    0.39,
    0.40,
    0.41,
    0.42,
    0.43,
    0.44,
    0.45,
    0.46,
    0.48,
    0.5,
    0.52,
    0.53,
    0.55,
    0.57,
    0.59,
    0.6,
    0.63,
    0.65,
    0.66,
    0.67,
    0.68,
    0.69,
    0.7,
    0.71,
    0.72,
    0.73,
    0.74,
    0.75,
    0.76,
    0.77,
    0.78,
    0.79,
    0.8,
    0.81,
    0.82,
    0.83,
    0.84,
    0.85,
]
colsample_bytree_range = [0.6, 0.7, 0.8, 0.9, 1.0]
num_rounds_grid = [130]

# Generate all combinations of hyperparameters
param_combinations = list(
    itertools.product(
        max_depth_range, learning_rate_range, colsample_bytree_range, num_rounds_grid
    )
)

# Construct the param_grid
param_grid = [
    {
        "objective": "reg:squarederror",
        "max_depth": max_depth,
        "learning_rate": learning_rate,
        "colsample_bytree": colsample_bytree,
        "n_estimators": n_estimators,
    }
    for max_depth, learning_rate, colsample_bytree, n_estimators in param_combinations
]


# Perform hyperparameter scan
experiment_hyperparams3 = experiment.xgboost_hyperparameter_scan(
    param_grid, nodes_info=True
)

In [ ]:
max_depth_range = [4, 5]
learning_rate_range = [0.74]
colsample_bytree_range = [1.0]
num_rounds_grid = [60, 65, 70, 75, 80, 85, 100, 130]

# Generate all combinations of hyperparameters
param_combinations = list(
    itertools.product(
        max_depth_range, learning_rate_range, colsample_bytree_range, num_rounds_grid
    )
)

# Construct the param_grid
param_grid = [
    {
        "objective": "reg:squarederror",
        "max_depth": max_depth,
        "learning_rate": learning_rate,
        "colsample_bytree": colsample_bytree,
        "n_estimators": n_estimators,
    }
    for max_depth, learning_rate, colsample_bytree, n_estimators in param_combinations
]


# Perform hyperparameter scan
experiment_hyperparams4 = experiment.xgboost_hyperparameter_scan(
    param_grid, nodes_info=True
)

In [ ]:
max_depth_range = [3, 4, 5, 6]
learning_rate_range = [0.74]
colsample_bytree_range = [1.0]
num_rounds_grid = [130, 150, 200, 275, 300, 600]

# Generate all combinations of hyperparameters
param_combinations = list(
    itertools.product(
        max_depth_range, learning_rate_range, colsample_bytree_range, num_rounds_grid
    )
)

# Construct the param_grid
param_grid = [
    {
        "objective": "reg:squarederror",
        "max_depth": max_depth,
        "learning_rate": learning_rate,
        "colsample_bytree": colsample_bytree,
        "n_estimators": n_estimators,
    }
    for max_depth, learning_rate, colsample_bytree, n_estimators in param_combinations
]


# Perform hyperparameter scan
experiment_hyperparams5 = experiment.xgboost_hyperparameter_scan(
    param_grid, nodes_info=True
)

In [ ]:
# train miniaturised model
print(experiment.X_test.shape)

params = {
    "objective": "reg:squarederror",
    "max_depth": 4,
    "learning_rate": 0.74,
    "gamma": 5000,  # min split loss
    "colsample_bytree": 1.0,
    "eval_metric": "rmse",
}

num_rounds = 130


experiment.train_xgboost_model(params, num_rounds)

In [ ]:
utils.count_nodes(experiment.model)

In [ ]:
mask_e = experiment.testing_dataset.physics_object_type == "electron"
mask_p = experiment.testing_dataset.physics_object_type == "photon"
mask_pi0 = experiment.testing_dataset.physics_object_type == "pi0"
mask_pi_char = experiment.testing_dataset.physics_object_type == "pi_char"

In [ ]:
x_e, y_e, x_u_e, u_e = ed.plot_avg(
    x_values=experiment.testing_dataset["et"].values[mask_e],
    y_values=(experiment.y_test - experiment.y_pred)[mask_e],
    interval=2000,
    xlabel="true et [MeV]",
    rms=True,
    return_values=True,
    ylabel="rms",
    return_x_u=True,
)

In [ ]:
x_p, y_p, x_u_p, u_p = ed.plot_avg(
    x_values=experiment.testing_dataset["et"].values[mask_p],
    y_values=(experiment.y_test - experiment.y_pred)[mask_p],
    interval=2000,
    xlabel="true et [MeV]",
    rms=True,
    ylabel="rms",
    return_values=True,
    return_x_u=True,
)

In [ ]:
x_pi0, y_pi0, x_u_pi0, u_pi0 = ed.plot_avg(
    x_values=experiment.testing_dataset["et"].values[mask_pi0],
    y_values=(experiment.y_test - experiment.y_pred)[mask_pi0],
    interval=2000,
    xlabel="true et [MeV]",
    rms=True,
    ylabel="rms",
    return_values=True,
    return_x_u=True,
)

In [ ]:
x_pi_char, y_pi_char, x_u_pi_char, u_pi_char = ed.plot_avg(
    x_values=experiment.testing_dataset["et"].values[mask_pi_char],
    y_values=(experiment.y_test - experiment.y_pred)[mask_pi_char],
    interval=2000,
    xlabel="true et [MeV]",
    rms=True,
    return_values=True,
    ylabel="rms",
    return_x_u=True,
)

In [ ]:
Y_test = experiment.y_test
Y_pred = experiment.y_pred
ed.plot_predictions(Y_test, Y_pred)

ed.plot_errors(Y_test, Y_pred, xmax=350, xcut=350, binnum=100)

ed.plot_corelation(Y_test, Y_pred, density=True, log_density=False, plot_line=True)